In [5]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [52]:
df = pd.read_csv('gastos.csv')
df = df[df['year'] == 2013]

In [40]:
df.head()

,year,sector name,entity name,item,cop,sector code,entity code
0,2013,AGROPECUARIO,AUTORIDAD NACIONAL DE ACUICULTURA Y PESCA - AUNAP,Funcionamiento,8.566545e+09,NaN,1715.0
1,2013,AGROPECUARIO,AUTORIDAD NACIONAL DE ACUICULTURA Y PESCA - AUNAP,Funcionamiento,8.566545e+09,NaN,1715.0
2,2013,AGROPECUARIO,AUTORIDAD NACIONAL DE ACUICULTURA Y PESCA - AUNAP,Inversión,1.136971e+10,NaN,1715.0
3,2013,AGROPECUARIO,AUTORIDAD NACIONAL DE ACUICULTURA Y PESCA - AUNAP,Inversión,1.136971e+10,NaN,1715.0
4,2013,AGROPECUARIO,INSTITUTO COLOMBIANO AGROPECUARIO (ICA),Funcionamiento,8.259477e+10,NaN,1702.0


In [41]:
dic_sectors = dict(enumerate(df['sector name'].unique()))
dic_entities = dict(enumerate(df['entity name'].unique()))
dic_items = dict(enumerate(df['item'].unique()))

In [42]:
dic_items

{0: 'Funcionamiento', 1: 'Inversión', 2: 'Servicio de la Deuda'}

In [43]:
reversed_sectors = {}
reversed_entities = {}
reversed_items = {}
for key, value in dic_sectors.items():
    reversed_sectors[value] = key

for key, value in dic_entities.items():
    reversed_entities[value] = key  + len(reversed_sectors)

for key, value in dic_items.items():
    reversed_items[value] = key  + len(reversed_sectors) + len(reversed_entities)

dic_info = reversed_sectors | reversed_entities | reversed_items



In [44]:
rel_sec_ent = (df
 .groupby(['sector name', 'entity name'])['cop']
 .sum()
 .reset_index()
 .rename(columns={'sector name': 'col0',
                  'entity name': 'col1'}))

In [45]:
rel_ent_it = (df
 .groupby(['entity name', 'item'])['cop']
 .sum()
 .reset_index()
 .rename(columns={'entity name': 'col0',
                  'item': 'col1'}))

In [46]:
rels = pd.concat([rel_sec_ent, rel_ent_it])
rels['col0'] = rels['col0'].map(dic_info)
rels['col1'] = rels['col1'].map(dic_info)

In [47]:
rels

,col0,col1,cop
0,0,29,3.987250e+10
1,0,30,5.244655e+11
2,0,31,4.627841e+11
3,0,32,1.595927e+12
4,0,33,4.735073e+11
...,...,...,...
326,153,217,5.648952e+10
327,147,216,9.693385e+11
328,147,217,1.262000e+10
329,174,216,1.217960e+11


In [49]:
fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = list(dic_info.keys()),
      color = "blue"
    ),
    link = dict(
      source = rels['col0'], # indices correspond to labels, eg A1, A2, A1, B1, ...
      target = rels['col1'],
      value = rels['cop']
  ))])

fig.update_layout(title_text="Basic Sankey Diagram", font_size=10)
fig.show()

In [53]:
import plotly.express as px
fig = px.treemap(df, path=[px.Constant("all"), 'sector name', 'entity name', 'item'], values='cop')
fig.update_traces(root_color="lightgrey")
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

In [51]:
df

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [2]:
import plotly.graph_objects as go 
import pandas as pd 
import json
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns


df = pd.read_csv('data/gastos_2013_2024_const.csv')








rel_sec_ent = (df
  .groupby(['year', 'sector_code', 'entidad'])['cop']
  .sum()
  .reset_index()
  .rename(columns={'sector_code': 'col0',
                    'entidad': 'col1'}))

rel_ent_it = (df
  .groupby(['year', 'entidad', 'cuenta'])['cop']
  .sum()
  .reset_index()
  .rename(columns={'entidad': 'col0',
                    'cuenta': 'col1'}))

rels = pd.concat([rel_sec_ent, rel_ent_it])

rels['col0'] = rels['col0'].map(dic_info)
rels['col1'] = rels['col1'].map(dic_info)

df = df[df['year'] == year]
rels = rels[rels['year'] == year]

df['PGN'] = len(df['sector_code'].unique()) + len(df['entidad'].unique())


fig = px.treemap(df, 
                  path=[px.Constant("PGN"), 'sector_code', 'entidad', 'cuenta'], 
                  values='cop',
                  width=1200,
                  height=800)
fig.update_traces(root_color="lightgrey")
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))


  
test1 = (df
          .groupby(['year', 'sector_code', 'cuenta'])['cop']
          .sum()
          .reset_index()
          .rename(columns={'sector_code':'col0',
                            'cuenta':'col1'}))

test2 = (df
          .groupby(['year','PGN', 'sector_code'])['cop']
          .sum()
          .reset_index()
          .rename(columns={'sector_code':'col1',
                            'PGN':'col0'}))

test = pd.concat([test1, test2], ignore_index=True)
test = test[test['year'] == year]


fig = go.Figure(data=[go.Sankey(
      node = dict(
        pad = 15,
        thickness = 20,
        line = dict(color = "black", width = 0.5),
        label = test['col0'].unique(),
        color = "blue"
      ),
      link = dict(
        source = test['col0'], # indices correspond to labels, eg A1, A2, A1, B1, ...
        target = test['col1'],
        value = test['cop']
    ))])

fig.update_layout(title_text="Presupuesto general", font_size=10, width=1000, height=700)
fig.show()

NameError: name 'dic_info' is not defined

In [20]:
dictio = {}
contador = 0
for i in df['cuenta'].unique():
    dictio[contador] = i
    contador += 1

for i in df['entidad'].unique():
    dictio[contador] = i 
    contador += 1

for i in df['sector_code'].unique():
    dictio[contador] = i
    contador += 1

reversed_dictio = {}
for key, value in dictio.items():
    try:
        value = int(value)
        print('converted')
    except:
        print("Cannot convert")
    finally: 
        reversed_dictio[value] = key

Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot convert
Cannot con

In [18]:
reversed_dictio

{'FUNCIONAMIENTO': 0,
 'INVERSION': 1,
 'DEUDA': 2,
 'CONGRESO DE LA REPÚBLICA': 3,
 'PRESIDENCIA DE LA REPÚBLICA': 4,
 'AGENCIA PRESIDENCIAL DE COOPERACIÓN INTERNACIONAL DE COLOMBIA, APC - COLOMBIA': 5,
 'UNIDAD NACIONAL PARA LA GESTIÓN DEL RIESGO DE DESASTRES': 6,
 'AGENCIA PARA LA REINCORPORACIÓN Y LA NORMALIZACIÓN - ARN': 7,
 'DEPARTAMENTO ADMINISTRATIVO NACIONAL DE PLANEACIÓN': 8,
 'UNIDAD ADMINISTRATIVA ESPECIAL - AGENCIA NACIONAL DE CONTRATACIÓN PÚBLICA - COLOMBIA COMPRA EFICIENTE': 9,
 'SUPERINTENDENCIA DE SERVICIOS PÚBLICOS DOMICILIARIOS': 10,
 'FONDO NACIONAL DE REGALÍAS EN LIQUIDACIÓN': 11,
 'DEPARTAMENTO ADMINISTRATIVO NACIONAL DE ESTADÍSTICA (DANE)': 12,
 'FONDO ROTATORIO DEL DANE': 13,
 'INSTITUTO GEOGRÁFICO AGUSTÍN CODAZZI - IGAC': 14,
 'DEPARTAMENTO ADMINISTRATIVO DE LA FUNCIÓN PÚBLICA': 15,
 'ESCUELA SUPERIOR DE ADMINISTRACIÓN PÚBLICA (ESAP) ': 16,
 'DEPARTAMENTO ADMINISTRATIVO DE SEGURIDAD (DAS) EN SUPRESIÓN': 17,
 'FONDO ROTATORIO DEL DEPARTAMENTO ADMINISTRATIVO DE S

In [19]:
import json 
with open('info.json', 'w') as js:
    json.dump(reversed_dictio, js) 

{'FUNCIONAMIENTO': 0,
 'INVERSION': 1,
 'DEUDA': 2,
 'CONGRESO DE LA REPÚBLICA': 3,
 'PRESIDENCIA DE LA REPÚBLICA': 4,
 'AGENCIA PRESIDENCIAL DE COOPERACIÓN INTERNACIONAL DE COLOMBIA, APC - COLOMBIA': 5,
 'UNIDAD NACIONAL PARA LA GESTIÓN DEL RIESGO DE DESASTRES': 6,
 'AGENCIA PARA LA REINCORPORACIÓN Y LA NORMALIZACIÓN - ARN': 7,
 'DEPARTAMENTO ADMINISTRATIVO NACIONAL DE PLANEACIÓN': 8,
 'UNIDAD ADMINISTRATIVA ESPECIAL - AGENCIA NACIONAL DE CONTRATACIÓN PÚBLICA - COLOMBIA COMPRA EFICIENTE': 9,
 'SUPERINTENDENCIA DE SERVICIOS PÚBLICOS DOMICILIARIOS': 10,
 'FONDO NACIONAL DE REGALÍAS EN LIQUIDACIÓN': 11,
 'DEPARTAMENTO ADMINISTRATIVO NACIONAL DE ESTADÍSTICA (DANE)': 12,
 'FONDO ROTATORIO DEL DANE': 13,
 'INSTITUTO GEOGRÁFICO AGUSTÍN CODAZZI - IGAC': 14,
 'DEPARTAMENTO ADMINISTRATIVO DE LA FUNCIÓN PÚBLICA': 15,
 'ESCUELA SUPERIOR DE ADMINISTRACIÓN PÚBLICA (ESAP) ': 16,
 'DEPARTAMENTO ADMINISTRATIVO DE SEGURIDAD (DAS) EN SUPRESIÓN': 17,
 'FONDO ROTATORIO DEL DEPARTAMENTO ADMINISTRATIVO DE S